# ML US baby names

## 1. Download raw data

In [1]:
from constants import *
print("STATES:", STATES)
print("YEAR_START:", YEAR_START)
print("YEAR_END:", YEAR_END)
print("URL_YEAR_COUNTRY:", URL_YEAR_COUNTRY)
print("URL_YEAR_STATE:", URL_YEAR_STATE)
print("RAW_CSV:", RAW_CSV)

STATES: ('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY')
YEAR_START: 1960
YEAR_END: 2021
URL_YEAR_COUNTRY: https://www.ssa.gov/cgi-bin/popularnames.cgi
URL_YEAR_STATE: https://www.ssa.gov/cgi-bin/namesbystate.cgi
RAW_CSV: data_raw_year_state.csv


In [2]:
import requests
import re
import json
import pandas

def clean_tags(html: str, tag: str):
    return re.sub(r"<" + tag + r" [^>]+>", "<" + tag + ">", html)

def extract_data_table(html: str):
    html = re.sub(r"\r", "", html)
    
    for part in html.split('<table '):
        if "Popularity for top 100 names" in part:
            table = "<table " + part.split("</table>")[0] + "</table>"
            
            table = clean_tags(table, "caption")
            table = table.replace("\n</caption>", "</caption>")
            table = re.sub(r"\n *<caption>.*<\/caption> *\n", "\n", table, flags=re.M)

            table = clean_tags(table, "br")
            table = re.sub(r"\s*<br>\s*", " ", table)

            for tag in ["table", "caption", "tr", "th", "td"]:
                table = clean_tags(table, tag)
            
            table = table.replace("&nbsp;", "0")
            
            return table
    
    raise Exception("Data table not found")

def table_to_json(html: str):
    return html.replace("<table>",  '[') \
               .replace("</table>", ']') \
               .replace("<tr>",  '[') \
               .replace("</tr>", '],') \
               .replace("<th>",  '"') \
               .replace("</th>", '",') \
               .replace("<td>",  '"') \
               .replace("</td>", '",')

def html_to_pandas(html: str):
    html_table = extract_data_table(html)
    data_json = table_to_json(html_table) \
                    .replace(",]", "]") \
                    .replace("],\n]", "]]")

    raw = json.loads(data_json)

    column_names = [x.lower().replace(" ", "_") for x in raw[0]]
    column_names

    data = []
    for row in raw[1:]:
        item = {}
        for i in range(0, len(column_names)):
            if column_names[i] != "rank":
                item[column_names[i]] = row[i]
        data.append(item)

    data = data[0:100]
        
    for i in range(0, len(data)):
        data[i]["number_of_males"] = int(data[i]["number_of_males"].replace(",", ""))
        data[i]["number_of_females"] = int(data[i]["number_of_females"].replace(",", ""))

    df = pandas.DataFrame(data)
    df.rename(columns = {'male_name':'m_name'}, inplace = True)
    df.rename(columns = {'number_of_males':'m_num'}, inplace = True)
    df.rename(columns = {'female_name':'f_name'}, inplace = True)
    df.rename(columns = {'number_of_females':'f_num'}, inplace = True)
    return df[["m_name", "m_num", "f_name", "f_num"]]

def fetch_data_per_year_state(year: int, state: str):
    params = {'year': year, 'state': state}
    r = requests.post(url = URL_YEAR_STATE, data = params)
    df = html_to_pandas(r.text)
    df.insert(loc=0, column='state', value=state)
    df.insert(loc=0, column='year', value=year)
    return df

In [3]:
print("Reading raw data, if it exists...")
print()

try:
    df = pandas.read_csv(RAW_CSV)
    print("Done.")
    print("Last row: %i, %s" % (df.iloc[-1]["year"], df.iloc[-1]["state"]))
    
except:
    df = None
    print("No data was found.")

df

Reading raw data, if it exists...

Done.
Last row: 2021, WY


,year,state,m_name,m_num,f_name,f_num
0,1960,AK,David,152,Mary,79
1,1960,AK,Michael,140,Linda,56
2,1960,AK,Robert,136,Karen,53
3,1960,AK,John,125,Debra,50
4,1960,AK,James,124,Susan,50
...,...,...,...,...,...,...
316195,2021,WY,Jacob,6,Gracelynn,5
316196,2021,WY,Jasper,6,Hadlee,5
316197,2021,WY,Luca,6,Isla,5
316198,2021,WY,Odin,6,Kendall,5


In [4]:
print("Downloading remaining per state and year from %i to %i..." % (YEAR_START, YEAR_END))
print()

for state in sorted(STATES):
    print(state + " ", end="")
    
    loaded = 0
    
    for year in range(YEAR_START, YEAR_END + 1):
        if df is None:
            df_current = fetch_data_per_year_state(year, state)
            print(".", end="")
        elif len(df[(df["year"] == year) & (df["state"] == state)]) > 0:
            print("-", end="")
            continue
        else:
            df_current = fetch_data_per_year_state(year, state)
            print(".", end="")
        
        loaded += 1
        
        if df is None:
            df = df_current
        else:
            df = pandas.concat([df, df_current])
        
    if loaded > 0:
        df.to_csv(RAW_CSV, index=False)

    print()

print()
print("Done.")
print('Data saved to "%s".' % RAW_CSV)


AK --------------------------------------------------------------
AL --------------------------------------------------------------
AR --------------------------------------------------------------
AZ --------------------------------------------------------------
CA --------------------------------------------------------------
CO --------------------------------------------------------------
CT --------------------------------------------------------------
DC --------------------------------------------------------------
DE --------------------------------------------------------------
FL --------------------------------------------------------------
GA --------------------------------------------------------------
HI --------------------------------------------------------------
IA --------------------------------------------------------------
ID --------------------------------------------------------------
IL --------------------------------------------------------------
IN ------

In [5]:
df

,year,state,m_name,m_num,f_name,f_num
0,1960,AK,David,152,Mary,79
1,1960,AK,Michael,140,Linda,56
2,1960,AK,Robert,136,Karen,53
3,1960,AK,John,125,Debra,50
4,1960,AK,James,124,Susan,50
...,...,...,...,...,...,...
316195,2021,WY,Jacob,6,Gracelynn,5
316196,2021,WY,Jasper,6,Hadlee,5
316197,2021,WY,Luca,6,Isla,5
316198,2021,WY,Odin,6,Kendall,5


In [6]:
print("Finished!")

Finished!
